Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'clean_notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['ctrain_dataset']
  train_labels = save['ctrain_labels']
  valid_dataset = save['cvalid_dataset']
  valid_labels = save['cvalid_labels']
  test_dataset = save['ctest_dataset']
  test_labels = save['ctest_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (187354, 28, 28) (187354,)
Validation set (8913, 28, 28) (8913,)
Test set (8629, 28, 28) (8629,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (187354, 784) (187354, 10)
Validation set (8913, 784) (8913, 10)
Test set (8629, 784) (8629, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [ ]:
import math
batch_size = 128

graph = tf.Graph()
with graph.as_default():
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  L2 = tf.nn.l2_loss(weights, name='L2') * 0.0075
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + L2
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [ ]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("L2: %f" % L2.eval())
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

In [ ]:
with tf.Session(graph=graph) as session:
    # sess.graph_def is the graph definition; that enables the Graph Visualizer.
    summary_writer = tf.train.SummaryWriter('/Volumes/darthDisk/machineLearning/udacity/logs_tf', session.graph)

In [ ]:
import math

batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size), name='train_ds')
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels), name='train_labels')
  tf_valid_dataset = tf.constant(valid_dataset, name='valid_ds')
  tf_test_dataset = tf.constant(test_dataset, name='test_ds')
    
  # Hidden layer
  weights_hidden = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]), name='h_weights')   
  biases_hidden = tf.Variable(tf.zeros([hidden_nodes]), name='h_biases')
  logits_hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden, name='h_logits') + biases_hidden, 
                             name='h_relu_logits')
    
  # outPut layer
  weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]), name='weights')
  biases = tf.Variable(tf.zeros([num_labels]), name='biases')
  logits = tf.matmul(logits_hidden, weights, name='logits') + biases
  
  # Training computation. || Combine the weights adding them !! ||
  L2 = ( tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(weights) ) * 0.00075
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + L2
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits, name='train_prediction')
  logits_hidden_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden)
  valid_prediction = tf.nn.softmax(tf.matmul(logits_hidden_valid, weights) + biases)
  logits_hidden_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden)
  test_prediction = tf.nn.softmax(tf.matmul(logits_hidden_test, weights) + biases)

In [ ]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size + 2)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print(L2.eval())  
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [ ]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size), name='train_ds')
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels), name='train_labels')
  tf_valid_dataset = tf.constant(valid_dataset, name='valid_ds')
  tf_test_dataset = tf.constant(test_dataset, name='test_ds')
    
  # Hidden layer
  weights_hidden = tf.Variable(tf.truncated_normal([image_size*image_size, hidden_nodes]), name='h_weights')   
  biases_hidden = tf.Variable(tf.zeros([hidden_nodes]), name='h_biases')
  logits_hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden, name='h_logits') + biases_hidden, 
                             name='h_relu_logits')

  # outPut layer
  weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]), name='weights')
  biases = tf.Variable(tf.zeros([num_labels]), name='biases')
  logits = tf.matmul(logits_hidden, weights, name='logits') + biases
  
  # Training computation. || Combine the weights adding them !! ||
  L2 = ( tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(weights) ) * 0.00075
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + L2
#   loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits, name='train_prediction')
  logits_hidden_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden)
  valid_prediction = tf.nn.softmax(tf.matmul(logits_hidden_valid, weights) + biases)
  logits_hidden_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden)
  test_prediction = tf.nn.softmax(tf.matmul(logits_hidden_test, weights) + biases)

In [ ]:
# Restricting training data to just a few batches!!
# train_dataset = train_dataset[:1000, :]
# train_labels = train_labels[:1000, :]

In [ ]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size + 1)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print(L2.eval())  
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [ ]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size), name='train_ds')
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels), name='train_labels')
  tf_valid_dataset = tf.constant(valid_dataset, name='valid_ds')
  tf_test_dataset = tf.constant(test_dataset, name='test_ds')
    
  # Hidden layer
  weights_hidden = tf.Variable(tf.truncated_normal([image_size*image_size, hidden_nodes]), name='h_weights')   
  biases_hidden = tf.Variable(tf.zeros([hidden_nodes]), name='h_biases')
  X = tf.matmul(tf_train_dataset, weights_hidden, name='h_logits') + biases_hidden
  logits_hidden = tf.nn.dropout(tf.nn.relu(X, name='h_relu_logits'), 0.5)
    
  # outPut layer
  weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]), name='weights')
  biases = tf.Variable(tf.zeros([num_labels]), name='biases')
  logits = tf.matmul(logits_hidden, weights, name='logits') + biases
  
  # Training computation. || Combine the weights adding them !! ||
  L2 = ( tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(weights) ) * 0.00075
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + L2
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
#   optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits, name='train_prediction')
  logits_hidden_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden)
  valid_prediction = tf.nn.softmax(tf.matmul(logits_hidden_valid, weights) + biases)
  logits_hidden_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden)
  test_prediction = tf.nn.softmax(tf.matmul(logits_hidden_test, weights) + biases)

In [ ]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size + 2)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print(L2.eval())  
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

In [ ]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size), name='train_ds')
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels), name='train_labels')
  tf_valid_dataset = tf.constant(valid_dataset, name='valid_ds')
  tf_test_dataset = tf.constant(test_dataset, name='test_ds')
    
  # Hidden layer
  weights_hidden = tf.Variable(tf.truncated_normal([image_size*image_size, hidden_nodes]), name='h_weights')   
  biases_hidden = tf.Variable(tf.zeros([hidden_nodes]), name='h_biases')
  X = tf.matmul(tf_train_dataset, weights_hidden, name='X') + biases_hidden
  logits_hidden = tf.nn.dropout(tf.nn.relu( X, name='h_relu_logits'), 0.19)

  # outPut layer
  weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]), name='weights')
  biases = tf.Variable(tf.zeros([num_labels]), name='biases')
  logits = tf.matmul(logits_hidden, weights, name='logits') + biases
  
  # Training computation. || Combine the weights adding them !! ||
  L2 = ( tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(weights) ) * 0.00075
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + L2
#   loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits, name='train_prediction')
  logits_hidden_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden)
  valid_prediction = tf.nn.softmax(tf.matmul(logits_hidden_valid, weights) + biases)
  logits_hidden_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden)
  test_prediction = tf.nn.softmax(tf.matmul(logits_hidden_test, weights) + biases)

In [ ]:
# Restricting training data to just a few batches!!
train_dataset = train_dataset[:1000, :]
train_labels = train_labels[:1000, :]

In [ ]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size + 1)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print(L2.eval())  
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [5]:
import math
num_steps = 19001

batch_size = 128
hidden_nodes_1 = 1024
hidden_nodes_2 = 300
hidden_nodes_3 = 50

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size*image_size), name='train_ds')
  tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels), name='train_labels')
  tf_valid_dataset = tf.constant(valid_dataset, name='valid_ds')
  tf_test_dataset = tf.constant(test_dataset, name='test_ds')
    
  # Hidden layer 1
  weights_hidden_1 = tf.Variable(tf.truncated_normal([image_size*image_size, hidden_nodes_1],
                                                      stddev= math.sqrt(2.0/(image_size*image_size))), 
                                                      name='h_weights_1')   
  
  biases_hidden_1 = tf.Variable(tf.zeros([hidden_nodes_1]), name='h_biases_1')
  X1 = tf.matmul(tf_train_dataset, weights_hidden_1, name='h_logits_1') + biases_hidden_1
  logits_hidden_1 = tf.nn.relu(X1, name='h_relu_logits_1')

  # Hidden layer 2
  weights_hidden_2 = tf.Variable(tf.truncated_normal([hidden_nodes_1, hidden_nodes_2],
                                                      stddev = math.sqrt(2.0/(hidden_nodes_1))), 
                                                      name ='h_weights_2')   
    
  biases_hidden_2 = tf.Variable(tf.zeros([hidden_nodes_2]), name='h_biases_2')
  X2 = tf.matmul(logits_hidden_1, weights_hidden_2, name='h_logits_2') + biases_hidden_2
  logits_hidden_2 = tf.nn.relu(X2, name='h_relu_logits_2')

  # Hidden layer 3
  weights_hidden_3 = tf.Variable(tf.truncated_normal([hidden_nodes_2, hidden_nodes_3],
                                                      stddev = math.sqrt(2.0/(hidden_nodes_2))),
                                                      name='h_weights_3')   

  biases_hidden_3 = tf.Variable(tf.zeros([hidden_nodes_3]), name='h_biases_3')
  X3 = tf.matmul(logits_hidden_2, weights_hidden_3, name='h_logits_3') + biases_hidden_3
  logits_hidden_3 = tf.nn.relu(X3, name='h_relu_logits_3')
    
  # outPut layer
  weights = tf.Variable(tf.truncated_normal([hidden_nodes_3, num_labels],
                                             stddev= math.sqrt(2.0/(hidden_nodes_3))), 
                                             name='weights_out')
    
  biases = tf.Variable(tf.zeros([num_labels]), name='biases_out')
  logits = tf.matmul(logits_hidden_3, weights, name='logits_out') + biases
  
  # Training computation. || Combine the weights adding them !! ||
  L2 = ( tf.nn.l2_loss(weights_hidden_1) + tf.nn.l2_loss(weights_hidden_2) +
         tf.nn.l2_loss(weights_hidden_3) + tf.nn.l2_loss(weights) ) * 0.00075
  
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + L2
    
  # Optimizer.
  global_step = tf.Variable(0) # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.7, global_step, num_steps*0.05, 0.90, staircase=False)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits, name='train_prediction')
  
  logits_hidden_valid_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden_1) + biases_hidden_1)
  logits_hidden_valid_2 = tf.nn.relu(tf.matmul(logits_hidden_valid_1, weights_hidden_2) + biases_hidden_2)
  logits_hidden_valid_3 = tf.nn.relu(tf.matmul(logits_hidden_valid_2, weights_hidden_3) + biases_hidden_3)
  valid_prediction = tf.nn.softmax(tf.matmul(logits_hidden_valid_3, weights) + biases)

  logits_hidden_test_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden_1) + biases_hidden_1)
  logits_hidden_test_2 = tf.nn.relu(tf.matmul(logits_hidden_test_1, weights_hidden_2) + biases_hidden_2)
  logits_hidden_test_3 = tf.nn.relu(tf.matmul(logits_hidden_test_2, weights_hidden_3) + biases_hidden_3)
  test_prediction = tf.nn.softmax(tf.matmul(logits_hidden_test_3, weights) + biases)

In [6]:
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:]
  shuffled_labels = labels[permutation,:]
  return shuffled_dataset, shuffled_labels

In [7]:
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size + 2)
    
    if offset < batch_size:
        train_dataset, train_labels = randomize(train_dataset, train_labels)
        print('shuffled all')
        
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print('L2: ', L2.eval())  
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
shuffled all
L2:  0.803205
Minibatch loss at step 0: 3.168500
Minibatch accuracy: 4.7%
Validation accuracy: 17.6%
L2:  0.535339
Minibatch loss at step 500: 1.124225
Minibatch accuracy: 82.8%
Validation accuracy: 85.0%
L2:  0.372429
Minibatch loss at step 1000: 0.747100
Minibatch accuracy: 89.8%
Validation accuracy: 86.7%
shuffled all
L2:  0.276918
Minibatch loss at step 1500: 0.705948
Minibatch accuracy: 86.7%
Validation accuracy: 86.5%
L2:  0.220737
Minibatch loss at step 2000: 0.557640
Minibatch accuracy: 88.3%
Validation accuracy: 87.5%
L2:  0.184847
Minibatch loss at step 2500: 0.567532
Minibatch accuracy: 85.2%
Validation accuracy: 87.3%
shuffled all
L2:  0.162899
Minibatch loss at step 3000: 0.442668
Minibatch accuracy: 91.4%
Validation accuracy: 88.3%
L2:  0.149617
Minibatch loss at step 3500: 0.487572
Minibatch accuracy: 88.3%
Validation accuracy: 88.7%
L2:  0.139677
Minibatch loss at step 4000: 0.558655
Minibatch accuracy: 87.5%
Validation accuracy: 88.5%
shuffled 

In [ ]:
with tf.Session(graph=graph) as session:
    # sess.graph_def is the graph definition; that enables the Graph Visualizer.
    summary_writer = tf.train.SummaryWriter('/Volumes/darthDisk/machineLearning/udacity/logs_tf', session.graph)